In [1]:
import sunpy.map
import sunpy.visualization.colormaps as cm
import sunpy.sun.constants
from sunpy.map.maputils import all_coordinates_from_map, coordinate_is_on_solar_disk
from aiapy.calibrate import register, update_pointing,normalize_exposure,correct_degradation
from aiapy.psf import deconvolve

import astropy.units as u
from astropy.constants import G, M_sun, R_sun, au
from astropy.time import Time
from astropy.coordinates import get_body_barycentric_posvel
from astropy.units import imperial
from astropy.stats import gaussian_fwhm_to_sigma

import scipy
from scipy import ndimage
import matplotlib.pyplot as plt
import plotly.tools as tls
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

import numpy as np
import pandas as pd
import math
from math import pi
import glob
import time
import datetime
import os
import importlib
import warnings

In [4]:
aia_files = sorted(glob.glob('.\\data\\AIA\\*image_lev1.fits'))
len(aia_files)

1460

In [5]:
aia_files

['.\\data\\AIA\\aia_lev1_304a_2010_06_01t00_00_08_12z_image_lev1.fits',
 '.\\data\\AIA\\aia_lev1_304a_2010_06_02t00_05_38_12z_image_lev1.fits',
 '.\\data\\AIA\\aia_lev1_304a_2010_06_03t00_00_02_13z_image_lev1.fits',
 '.\\data\\AIA\\aia_lev1_304a_2010_06_04t00_00_02_13z_image_lev1.fits',
 '.\\data\\AIA\\aia_lev1_304a_2010_06_05t00_00_05_50z_image_lev1.fits',
 '.\\data\\AIA\\aia_lev1_304a_2010_06_06t00_00_05_50z_image_lev1.fits',
 '.\\data\\AIA\\aia_lev1_304a_2010_06_07t00_00_05_50z_image_lev1.fits',
 '.\\data\\AIA\\aia_lev1_304a_2010_06_08t00_00_05_50z_image_lev1.fits',
 '.\\data\\AIA\\aia_lev1_304a_2010_06_09t00_00_05_50z_image_lev1.fits',
 '.\\data\\AIA\\aia_lev1_304a_2010_06_10t00_00_05_50z_image_lev1.fits',
 '.\\data\\AIA\\aia_lev1_304a_2010_06_11t00_00_02_12z_image_lev1.fits',
 '.\\data\\AIA\\aia_lev1_304a_2010_06_12t00_00_02_13z_image_lev1.fits',
 '.\\data\\AIA\\aia_lev1_304a_2010_06_13t00_00_02_12z_image_lev1.fits',
 '.\\data\\AIA\\aia_lev1_304a_2010_06_14t00_00_02_12z_image_lev1

In [9]:
import warnings
warnings.filterwarnings("ignore")
#用aiapy处理sunpy map 得到 level 1.5 的数据，并存成"_adjusted.fits"
error_files=[]
for i in list(range(len(aia_files)))[:3]:
    
    try:       
        # Save the adjusted map to a new file
        new_file=aia_files[i].split(sep='\\')
        new_file[2]='AIA_1p5'
        new_file[-1]=new_file[-1].replace('.fits', '_adjusted.fits')         
        new_file= os.path.join(*new_file)
        
        
        if(os.path.exists(new_file)):
            print("exist    "+new_file)
            continue
        
    
        # rp=[original_map.meta['crpix1'],original_map.meta['crpix2']]
        # image_shape=[original_map.meta['naxis1'],original_map.meta['naxis2']]
        # if(
        # ((rp[0]-image_shape[0]/2)**2+(rp[1]-image_shape[1]/2)**2)/(image_shape[0]*image_shape[1]/4)>0.05
        # ):
        #     print("not center "+new_file)
        #     continue
        

            
        # Load the original map from a file
        smap = sunpy.map.Map(aia_files[i])
        
        # Adjust the map using aiapy functions
        smap=deconvolve(smap, clip_negative=True,use_gpu=True)
        smap=update_pointing(smap)
        smap=register(smap)
        smap=correct_degradation(smap)
        adjusted_map=normalize_exposure(smap)
        adjusted_map.save(new_file, overwrite=False)
         
        if(i%10==0):
            print(f"Saved adjusted map to {new_file}")

    except:
        error_files.append(aia_files[i])
        print(f"Error  file {aia_files[i]}")

exist    .\data\AIA_1p5\aia_lev1_304a_2010_06_01t00_00_08_12z_image_lev1_adjusted.fits
exist    .\data\AIA_1p5\aia_lev1_304a_2010_06_02t00_05_38_12z_image_lev1_adjusted.fits
exist    .\data\AIA_1p5\aia_lev1_304a_2010_06_03t00_00_02_13z_image_lev1_adjusted.fits
